In [1]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

In [2]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


In [3]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns={'sales_ymd':'sales_date'}).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


In [4]:
df_receipt[df_receipt['customer_id']=='CS018205000001'].loc[:,['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [5]:
df_receipt[(df_receipt['customer_id']=='CS018205000001') & (df_receipt['amount']>=1000)].loc[:,['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [6]:
df_receipt[(df_receipt['customer_id']=='CS018205000001') & ((df_receipt['amount']>=1000) | (df_receipt['quantity'] >= 5))].loc[:,['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [7]:
df_receipt[(df_receipt['customer_id']=='CS018205000001') & (df_receipt['amount']>=1000) & (df_receipt['amount']<=2000)].loc[:,['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


In [8]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" & product_cd!="P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [9]:
df_store.query('prefecture_cd != "13" & floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [10]:
df_store.query('store_cd.astype("str").str.startswith("S14")', engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


In [11]:
df_customer.query('customer_id.str.endswith("1")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


In [12]:
df_store.query('address.str.contains("横浜市")', engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


In [13]:
df_customer.query('status_cd.str.match(".*[0-9]$")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
5,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0
7,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0
8,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
10,CS007403000016,依田 満,0,男性,1975-08-18,43,276-0022,千葉県八千代市上高野**********,S12007,20150914,0-00000000-0
11,CS035614000014,板倉 菜々美,1,女性,1954-07-16,64,154-0015,東京都世田谷区桜新町**********,S13035,20150804,0-00000000-0


In [14]:
df_customer.query('status_cd.str.match(".*[A-F]$")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
49,CS040415000104,金井 由美子,1,女性,1971-03-14,48,226-0026,神奈川県横浜市緑区長津田町**********,S14040,20160306,7-20090423-A
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


In [15]:
df_customer.query('status_cd.str.match("^[A-F].*[1-9]$")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


In [16]:
df_store.query('tel_no.str.match("^\d{3}-\d{3}-\d{4}$")', engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


In [17]:
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


In [18]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


In [19]:
df_tmp=pd.concat([df_receipt[['customer_id', 'amount']], df_receipt['amount'].rank(ascending=False, method='min')], axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('amount', ascending=False).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
72747,ZZ000000000000,5480,4.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,4.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,7.0
77874,CS008513000063,5280,9.0
55795,CS032414000072,5280,9.0


In [20]:
df_tmp=pd.concat([df_receipt[['customer_id', 'amount']], df_receipt['amount'].rank(ascending=False, method='first')], axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


In [21]:
len(df_receipt)

104681

In [22]:
len(df_receipt['customer_id'].unique())

8307

In [23]:
df_receipt.groupby('store_cd').agg({'amount':'sum', 'quantity':'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


In [24]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


In [25]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


In [26]:
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.columns = ['_'.join(s) for s in df_tmp.columns]
df_tmp.query('sales_ymd_max!=sales_ymd_min').head(10)

,customer_id_,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


In [27]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


In [28]:
df_receipt.groupby('store_cd').amount.median().reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


In [29]:
df_receipt.groupby('store_cd').amount.apply(lambda x: x.mode()).reset_index()

,store_cd,level_1,amount
0,S12007,0,148
1,S12013,0,148
2,S12014,0,100
3,S12029,0,100
4,S12030,0,148
5,S13001,0,100
6,S13002,0,100
7,S13003,0,1100
8,S13004,0,98
9,S13005,0,88


In [30]:
df_receipt.groupby('store_cd').amount.var().reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,441863.252526
31,S14011,306442.242432
42,S14034,297068.392740
5,S13001,295558.842618
12,S13015,295427.197086


In [31]:
df_receipt.groupby('store_cd').amount.std().reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,664.727954
31,S14011,553.572256
42,S14034,545.039808
5,S13001,543.653237
12,S13015,543.532149


In [32]:
df_receipt.amount.quantile(q=np.arange(5)/4)

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

In [33]:
np.arange(5)/4

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [34]:
df_receipt.groupby('store_cd').agg({'amount': 'mean'}).reset_index().query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


In [35]:
df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().mean()

2547.742234529256

In [36]:
average = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().mean()
df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
groupby('customer_id').agg({'amount': 'sum'}).query('amount >= '+ str(average)).head(10).reset_index()

,customer_id,amount
0,CS001115000010,3044
1,CS001205000006,3337
2,CS001214000009,4685
3,CS001214000017,4132
4,CS001214000052,5639
5,CS001215000040,3496
6,CS001304000006,3726
7,CS001305000005,3485
8,CS001305000011,4370
9,CS001315000180,3300


In [37]:
pd.merge(df_receipt, df_store[['store_name', 'store_cd']], how='inner', on='store_cd').head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店


In [38]:
pd.merge(df_category[['category_small_name', 'category_small_cd']], df_product, how='inner', on='category_small_cd').head(10)

,category_small_name,category_small_cd,product_cd,category_major_cd,category_medium_cd,unit_price,unit_cost
0,弁当類,040101,P040101001,04,0401,198.0,149.0
1,弁当類,040101,P040101002,04,0401,218.0,164.0
2,弁当類,040101,P040101003,04,0401,230.0,173.0
3,弁当類,040101,P040101004,04,0401,248.0,186.0
4,弁当類,040101,P040101005,04,0401,268.0,201.0
5,弁当類,040101,P040101006,04,0401,298.0,224.0
6,弁当類,040101,P040101007,04,0401,338.0,254.0
7,弁当類,040101,P040101008,04,0401,420.0,315.0
8,弁当類,040101,P040101009,04,0401,498.0,374.0
9,弁当類,040101,P040101010,04,0401,580.0,435.0


In [39]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [40]:
df_tmp = df_customer.query('(not customer_id.str.startswith("Z")) and (gender_cd.str.contains("1"))', engine='python')
amount_sum = df_receipt.groupby('customer_id').amount.sum()
pd.merge(df_tmp['customer_id'], amount_sum, how='left', on='customer_id').fillna(0).head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


In [41]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [42]:
df_tmp=df_receipt.query('not customer_id.str.startswith("Z")', engine='python')
df_sum=df_tmp.groupby('customer_id').amount.sum().reset_index().sort_values('amount', ascending=False).head(20)
df_cnt=df_tmp[~df_tmp.duplicated(subset=['sales_ymd', 'customer_id'])].groupby('customer_id').sales_ymd.count().\
reset_index().sort_values('sales_ymd', ascending=False).head(20)

pd.merge(df_sum, df_cnt, how='outer', on='customer_id')

,customer_id,amount,sales_ymd
0,CS017415000097,23086.0,20.0
1,CS015415000185,20153.0,22.0
2,CS031414000051,19202.0,19.0
3,CS028415000007,19127.0,21.0
4,CS001605000009,18925.0,NaN
5,CS010214000010,18585.0,22.0
6,CS016415000141,18372.0,20.0
7,CS006515000023,18372.0,NaN
8,CS011414000106,18338.0,NaN
9,CS038415000104,17847.0,NaN


In [43]:
len(df_store)*len(df_product)

531590

In [44]:
df_receipt.groupby('sales_ymd').amount.sum().diff().reset_index()[1:].head(10)

,sales_ymd,amount
1,20170102,-9558.0
2,20170103,3338.0
3,20170104,8662.0
4,20170105,1665.0
5,20170106,-5443.0
6,20170107,-8972.0
7,20170108,1322.0
8,20170109,1981.0
9,20170110,-6575.0
10,20170111,4144.0


In [45]:
df_zero = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_one = df_zero.groupby('sales_ymd').amount.sum().reset_index().shift(1)
df_one.columns=['sales_ymd_one', 'lag_one']
df_two = df_zero.groupby('sales_ymd').amount.sum().reset_index().shift(2)
df_two.columns=['sales_ymd_tho', 'lag_two']
df_three = df_zero.groupby('sales_ymd').amount.sum().reset_index().shift(3)
df_three.columns=['sales_ymd_three', 'lag_three']
pd.concat([df_zero, df_one, df_two, df_three],axis=1).dropna().head()

,sales_ymd,amount,sales_ymd_one,lag_one,sales_ymd_tho,lag_two,sales_ymd_three,lag_three
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0


In [46]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [47]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [48]:
df_tmp = pd.merge(df_customer[['customer_id','gender_cd', 'age']], 
                  df_receipt[['customer_id', 'amount']], how='inner', on='customer_id')
df_tmp['ages'] = (df_tmp['age'] // 10) * 10
df_tmp = df_tmp.groupby(['gender_cd', 'ages']).amount.sum().reset_index()
df_sales_summary = pd.pivot_table(df_tmp, index='ages',
                                  columns='gender_cd',values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


In [49]:
dss_v2=df_sales_summary.set_index("era")
dss_v2 = dss_v2.stack().reset_index().replace({'male':'00',
                                               'female':'01', 'unknown':'99'}).rename(columns={'level_1':'gender', 0: 'amount'})
dss_v2

,era,gender,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


In [50]:
dss_v2

,era,gender,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


In [51]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')], axis=1).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


In [52]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['application_date'], format='%Y%m%d')], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


In [53]:
pd.concat([df_receipt['receipt_no'], df_receipt['receipt_sub_no'], pd.to_datetime(df_receipt['sales_ymd'].astype(str))], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


In [54]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [55]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s')], axis=1)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
...,...,...,...
104676,1132,2,2018-02-21
104677,1132,2,2019-09-11
104678,1122,1,2017-03-11
104679,1142,1,2017-03-31


In [56]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch']).dt.year], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,1970
1,1132,2,1970
2,1102,1,1970
3,1132,1,1970
4,1102,2,1970
5,1112,1,1970
6,1102,2,1970
7,1102,1,1970
8,1112,2,1970
9,1102,1,1970


In [57]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch']).dt.strftime('%2m')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,01
1,1132,2,01
2,1102,1,01
3,1132,1,01
4,1102,2,01
5,1112,1,01
6,1102,2,01
7,1102,1,01
8,1112,2,01
9,1102,1,01


In [58]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine="python")
df_tmp = df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp['amount'] = (df_tmp['amount'] >= 2000).astype(int)
df_tmp.head(10)

,customer_id,amount
0,CS001113000004,0
1,CS001114000005,0
2,CS001115000010,1
3,CS001205000004,0
4,CS001205000006,1
5,CS001211000025,0
6,CS001212000027,0
7,CS001212000031,0
8,CS001212000046,0
9,CS001212000070,0


In [59]:
df_tmp = df_customer.copy()
df_tmp['postal_flg'] = np.where(df_tmp['postal_cd'].str.match("^1[0-9][0-9]|20[0-9].*"), 1, 0)
df_v2 = pd.merge(df_receipt, df_tmp, how='inner', on='customer_id')
df_v2 = df_v2[~df_v2.duplicated('customer_id')]
df_v2.groupby('postal_flg').customer_id.count()

postal_flg
0    3906
1    4400
Name: customer_id, dtype: int64

In [60]:
df_customer.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
5,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
7,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0
8,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5


In [61]:
pd.concat([df_customer[['address', 'customer_id']], df_customer['address'].str[0:3].map({'埼玉県': '11',
                                                '千葉県':'12', 
                                                '東京都':'13', 
                                                '神奈川':'14'})], axis=1).head()

,address,customer_id,address
0,神奈川県伊勢原市粟窪**********,CS021313000114,14
1,東京都江東区南砂**********,CS037613000071,13
2,東京都渋谷区代々木**********,CS031415000172,13
3,神奈川県横浜市泉区和泉町**********,CS028811000001,14
4,東京都大田区仲六郷**********,CS001215000145,13


In [62]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
quant = df_tmp.amount.quantile([0.25, 0.5, 0.75])
def classify(x):
    if x >= quant[0.75]:
        return 4
    if x >= quant[0.50]:
        return 3
    if x >= quant[0.25]:
        return 2
    return 1

df_tmp['class'] = df_tmp['amount'].apply(lambda x: classify(x))
df_tmp.head(10)

,customer_id,amount,class
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


In [63]:
df_tmp = df_customer[['age', 'customer_id', 'birth_day']].copy()
def get_era(x):
    era = x // 10 * 10
    if era >= 60:
        era = 60
    return era

df_tmp['era'] = df_tmp.age.apply(lambda x: get_era(x))
df_tmp.head()
 ## min(60, floor(x / 10) * 10)

,age,customer_id,birth_day,era
0,37,CS021313000114,1981-04-29,30
1,66,CS037613000071,1952-04-01,60
2,42,CS031415000172,1976-10-04,40
3,86,CS028811000001,1933-03-27,60
4,24,CS001215000145,1995-03-29,20


In [64]:
df_tmp['era_gender'] = df_tmp['era'] + df_customer['gender_cd'].astype(int)*100
df_tmp.head(10)

,age,customer_id,birth_day,era,era_gender
0,37,CS021313000114,1981-04-29,30,130
1,66,CS037613000071,1952-04-01,60,960
2,42,CS031415000172,1976-10-04,40,140
3,86,CS028811000001,1933-03-27,60,160
4,24,CS001215000145,1995-03-29,20,120
5,44,CS020401000016,1974-09-15,40,40
6,41,CS015414000103,1977-08-09,40,140
7,45,CS029403000008,1973-08-17,40,40
8,87,CS015804000004,1931-05-02,60,60
9,56,CS033513000180,1962-07-11,50,150


In [65]:
pd.concat([pd.get_dummies(df_customer['gender_cd']), df_customer['customer_id']], axis=1).head(10)

,0,1,9,customer_id
0,0,1,0,CS021313000114
1,0,0,1,CS037613000071
2,0,1,0,CS031415000172
3,0,1,0,CS028811000001
4,0,1,0,CS001215000145
5,1,0,0,CS020401000016
6,0,1,0,CS015414000103
7,1,0,0,CS029403000008
8,1,0,0,CS015804000004
9,0,1,0,CS033513000180


In [66]:
pd.get_dummies(df_customer[['gender_cd', 'customer_id']], columns=['gender_cd'])

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
...,...,...,...,...
21966,CS002512000474,0,1,0
21967,CS029414000065,0,1,0
21968,CS012403000043,1,0,0
21969,CS033512000184,0,1,0


In [67]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
    copy().groupby('customer_id').amount.sum().reset_index()
ave = df_tmp.amount.mean()
std = np.var(df_tmp.amount) ** 0.5
df_tmp['std'] = df_tmp.amount.apply(lambda x: (x - ave) / std)
df_tmp.head()

,customer_id,amount,std
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114


In [68]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
    groupby('customer_id').amount.sum().reset_index()
df_tmp['log_amount'] = df_tmp.amount.apply(lambda x: np.log10(x + 1))
df_tmp.head()

,customer_id,amount,log_amount
0,CS001113000004,1298,3.113609
1,CS001114000005,626,2.797268
2,CS001115000010,3044,3.483587
3,CS001205000004,1988,3.298635
4,CS001205000006,3337,3.523486


In [69]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
    groupby('customer_id').amount.sum().reset_index()
df_tmp['log_amount'] = df_tmp.amount.apply(lambda x: np.log(x + 1))
df_tmp.head()

,customer_id,amount,log_amount
0,CS001113000004,1298,7.169350
1,CS001114000005,626,6.440947
2,CS001115000010,3044,8.021256
3,CS001205000004,1988,7.595387
4,CS001205000006,3337,8.113127


In [70]:
df_tmp = df_product[['product_cd', 'unit_price', 'unit_cost']].copy()
df_tmp['profit'] = df_tmp['unit_price'] - df_tmp.unit_cost
df_tmp.head(10)

,product_cd,unit_price,unit_cost,profit
0,P040101001,198.0,149.0,49.0
1,P040101002,218.0,164.0,54.0
2,P040101003,230.0,173.0,57.0
3,P040101004,248.0,186.0,62.0
4,P040101005,268.0,201.0,67.0
5,P040101006,298.0,224.0,74.0
6,P040101007,338.0,254.0,84.0
7,P040101008,420.0,315.0,105.0
8,P040101009,498.0,374.0,124.0
9,P040101010,580.0,435.0,145.0


In [71]:
prof_rate = (df_product['unit_price'] - df_product['unit_cost']) / df_product['unit_price']
np.mean(prof_rate)

0.24911389885176904

In [72]:
newprice = np.floor(df_product['unit_cost'].dropna() * 1.3)
(newprice - df_product['unit_cost'].dropna()) / df_product['unit_cost'].dropna()


0        0.295302
1        0.298780
2        0.294798
3        0.295699
4        0.298507
           ...   
10025    0.300000
10026    0.300000
10027    0.300000
10028    0.299528
10029    0.300000
Name: unit_cost, Length: 10023, dtype: float64

In [73]:
newprice = np.ceil(df_product['unit_cost'].dropna() * 1.3)
(newprice - df_product['unit_cost'].dropna()) / df_product['unit_cost'].dropna()

0        0.302013
1        0.304878
2        0.300578
3        0.301075
4        0.303483
           ...   
10025    0.300000
10026    0.300000
10027    0.300000
10028    0.300708
10029    0.300000
Name: unit_cost, Length: 10023, dtype: float64

In [74]:
tax_price = np.floor(df_product['unit_price'].dropna() * 1.1)
tax_price.head()

0    217.0
1    239.0
2    253.0
3    272.0
4    294.0
Name: unit_price, dtype: float64

In [75]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [76]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,04,0401,040101,198.0,149.0
1,P040101002,04,0401,040101,218.0,164.0
2,P040101003,04,0401,040101,230.0,173.0
3,P040101004,04,0401,040101,248.0,186.0
4,P040101005,04,0401,040101,268.0,201.0


In [77]:
df_tmp_1 = pd.merge(df_receipt, df_product, how='inner', on='product_cd').groupby('customer_id').agg({'amount': 'sum'}).reset_index()
df_tmp_2 = pd.merge(df_receipt, df_product.query('category_major_cd == "07"'), how='inner', on='product_cd').groupby('customer_id').agg({'amount': 'sum'}).reset_index()
df_tmp = pd.merge(df_tmp_1, df_tmp_2, how='inner', on='customer_id')
df_tmp['rate_07'] = df_tmp['amount_y'] / df_tmp['amount_x']
df_tmp.head(10)


,customer_id,amount_x,amount_y,rate_07
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


In [78]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [79]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [80]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp['pass_time'] = (pd.to_datetime(df_tmp['application_date']) - pd.to_datetime(df_tmp['sales_ymd'].astype(str))).apply(lambda x: -x.days)
df_tmp.head()

,customer_id,sales_ymd,application_date,pass_time
0,CS006214000001,20181103,20150201,1371
1,CS006214000001,20170509,20150201,828
2,CS006214000001,20170608,20150201,858
3,CS006214000001,20170608,20150201,858
4,CS006214000001,20181028,20150201,1365


In [81]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp['pass_time'] = (pd.to_datetime(df_tmp['application_date']).apply(lambda x: x.replace(day=1))\
                       - pd.to_datetime(df_tmp['sales_ymd'].astype(str)).apply(lambda x: x.replace(day=1))).apply(lambda x: -x.days)
df_tmp.head()

,customer_id,sales_ymd,application_date,pass_time
0,CS006214000001,20181103,20150201,1369
1,CS006214000001,20170509,20150201,820
2,CS006214000001,20170608,20150201,851
3,CS006214000001,20170608,20150201,851
4,CS006214000001,20181028,20150201,1338


In [82]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']],
                df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['application_date']=pd.to_datetime(df_tmp['application_date'])
df_tmp['sales_ymd']=pd.to_datetime(df_tmp['sales_ymd'].astype(str))

df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']]. \
    apply(lambda x: relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis = 1)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,45
1,CS006214000001,2017-05-09,2015-02-01,27
2,CS006214000001,2017-06-08,2015-02-01,28
4,CS006214000001,2018-10-28,2015-02-01,44
7,CS006214000001,2019-09-08,2015-02-01,55
8,CS006214000001,2018-01-31,2015-02-01,35
9,CS006214000001,2017-07-05,2015-02-01,29
10,CS006214000001,2018-11-10,2015-02-01,45
12,CS006214000001,2019-04-10,2015-02-01,50
15,CS006214000001,2019-06-01,2015-02-01,52


In [83]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']],
                df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['application_date']=pd.to_datetime(df_tmp['application_date'])
df_tmp['sales_ymd']=pd.to_datetime(df_tmp['sales_ymd'].astype(str))

df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']]. \
    apply(lambda x: relativedelta(x[0], x[1]).years, axis = 1)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2
10,CS006214000001,2018-11-10,2015-02-01,3
12,CS006214000001,2019-04-10,2015-02-01,4
15,CS006214000001,2019-06-01,2015-02-01,4


In [84]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']],
                df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['application_date']=pd.to_datetime(df_tmp['application_date'])
df_tmp['sales_ymd']=pd.to_datetime(df_tmp['sales_ymd'].astype(str))

df_tmp['elapsed_date'] = df_tmp[['sales_ymd', 'application_date']]. \
    apply(lambda x: x[0].timestamp() - x[1].timestamp(), axis = 1)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,elapsed_date
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


In [85]:
df_tmp=pd.merge(df_receipt[['customer_id', 'sales_ymd']],
                df_customer[['application_date', 'customer_id']], how='inner', on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd']=pd.to_datetime(df_tmp['sales_ymd'].astype(str))

df_tmp['monday'] = df_tmp['sales_ymd']. \
    apply(lambda x: x - relativedelta(days=x.weekday()))
df_tmp['elapsed_weekdays'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp.head()

,customer_id,sales_ymd,application_date,monday,elapsed_weekdays
0,CS006214000001,2018-11-03,20150201,2018-10-29,5 days
1,CS006214000001,2017-05-09,20150201,2017-05-08,1 days
2,CS006214000001,2017-06-08,20150201,2017-06-05,3 days
4,CS006214000001,2018-10-28,20150201,2018-10-22,6 days
7,CS006214000001,2019-09-08,20150201,2019-09-02,6 days


In [86]:
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
17438,CS003213000021,布施 綾,1,女性,1991-09-02,27,201-0012,東京都狛江市中和泉**********,S13003,20150312,0-00000000-0
10415,CS005415000097,長田 郁恵,1,女性,1973-06-26,45,167-0022,東京都杉並区下井草**********,S13005,20141029,D-20100622-D
10647,CS003314000127,河野 早紀,1,女性,1987-10-26,31,201-0012,東京都狛江市中和泉**********,S13003,20160727,2-20100924-2
19673,CS003304000021,岩村 崇史,0,男性,1987-04-25,31,201-0012,東京都狛江市中和泉**********,S13003,20170221,1-20091020-2
16737,CS001403000132,根本 ジローラモ,0,男性,1969-03-31,50,144-0056,東京都大田区西六郷**********,S13001,20170725,0-00000000-0
13046,CS020413000214,深谷 理紗,1,女性,1976-08-27,42,173-0003,東京都板橋区加賀**********,S13020,20150618,0-00000000-0
2637,CS025415000005,市川 未華子,1,女性,1970-06-24,48,242-0024,神奈川県大和市福田**********,S14025,20150429,A-20100317-9
17098,CS007512000177,熊沢 陽子,1,女性,1966-07-07,52,276-0028,千葉県八千代市村上**********,S12007,20151130,0-00000000-0
20889,CS020615000032,岩本 翔子,1,女性,1955-05-09,63,173-0001,東京都板橋区本町**********,S13020,20150613,0-00000000-0
5470,CS001501000012,久保田 ひとり,0,男性,1962-04-29,56,212-0004,神奈川県川崎市幸区小向西町**********,S13001,20150917,0-00000000-0


In [87]:
_, df_tmp = train_test_split(df_customer, test_size=0.1, 
                             stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})

,customer_id
gender_cd,
0,298
1,1793
9,107


In [88]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.query('abs(amount_ss) >= 3').head()

,customer_id,amount,amount_ss
332,CS001605000009,18925,6.019921
1755,CS006415000147,12723,3.740202
1817,CS006515000023,18372,5.816651
1833,CS006515000125,12575,3.685800
1841,CS006515000209,11373,3.243972


In [89]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').\
groupby('customer_id').amount.sum()
quant = df_tmp.quantile([0.25, 0.75])
iqr = quant[0.75] - quant[0.25]
limit_down = quant[0.25] - iqr * 1.5
limit_upper = quant[0.75] + iqr * 1.5
df_tmp.reset_index().query('amount > '+ str(limit_upper) + 'or amount < ' + str(limit_down)).head()

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734


In [90]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

In [91]:
df_product_1 = df_product.dropna()
print(len(df_product))
print(len(df_product_1))

10030
10023


In [92]:
df_product_2 = df_product.fillna({
    'unit_price': np.round(np.nanmean(df_product['unit_price'])),
    'unit_cost': np.round(np.nanmean(df_product['unit_cost']))
})
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

In [93]:
df_product_3 = df_product.fillna({
    'unit_price': np.round(np.nanmedian(df_product['unit_price'])),
    'unit_cost': np.round(np.nanmedian(df_product['unit_cost']))
})
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

In [94]:
df_tmp = df_product.groupby('category_small_cd'). \
    agg({'unit_price': 'median', 'unit_cost': 'median'}).reset_index()
df_tmp.columns = ['category_small_cd', 'unit_price_median', 'unit_cost_median']
df_product_4 = pd.merge(df_product[['category_small_cd', 'unit_price', 'unit_cost']],
                       df_tmp, how='inner', on='category_small_cd')
df_product_4['unit_price'] = df_product_4[['unit_price', 'unit_price_median']].apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else np.round(x[0]), axis=1)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'unit_cost_median']].apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else np.round(x[0]), axis=1)
df_product_4.head()

,category_small_cd,unit_price,unit_cost,unit_price_median,unit_cost_median
0,040101,198.0,149.0,283.0,212.5
1,040101,218.0,164.0,283.0,212.5
2,040101,230.0,173.0,283.0,212.5
3,040101,248.0,186.0,283.0,212.5
4,040101,268.0,201.0,283.0,212.5


In [95]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90


In [96]:
df_tmp_0 = df_receipt.query('20190101 <= sales_ymd <= 20191231')
df_tmp_1 = pd.merge(df_customer['customer_id'],\
                    df_tmp_0[['customer_id', 'amount']],\
                    how='left', on='customer_id').groupby('customer_id').amount.sum().reset_index().rename(columns={'amount':'amount_2019'})

df_tmp_2 = pd.merge(df_customer['customer_id'], df_receipt[['customer_id', 'amount']],
                   how='left', on='customer_id').groupby('customer_id').amount.sum().reset_index()

df_tmp3 = pd.merge(df_tmp_1, df_tmp_2, how='inner', on='customer_id').fillna(0)
df_tmp3.query('amount_2019 > 0').head(10)

,customer_id,amount_2019,amount
8,CS001113000004,1298.0,1298.0
10,CS001114000005,188.0,626.0
12,CS001115000010,578.0,3044.0
17,CS001205000004,702.0,1988.0
18,CS001205000006,486.0,3337.0
23,CS001211000025,456.0,456.0
30,CS001212000070,456.0,456.0
57,CS001214000009,664.0,4685.0
59,CS001214000017,2962.0,4132.0
61,CS001214000048,1889.0,2374.0


In [97]:
df_tmp=df_geocode.groupby('postal_cd').agg({'longitude':'mean', 'latitude': 'mean'}).reset_index()
df_customer_1 = pd.merge(df_customer, df_tmp, how='left', on='postal_cd').\
rename(columns={'longitude':'longitude_m', 'latitude':'latitude_m', 'address':'address_m', 'application_store_cd':'store_cd'})
df_customer_1.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address_m,store_cd,application_date,status_cd,longitude_m,latitude_m
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.68965,35.67374
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.48360,35.39125
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.70775,35.54084


In [98]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [99]:
# コード例1
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) 
                    * math.sin(math.radians(y2)) 
                    + math.cos(math.radians(y1)) 
                    * math.cos(math.radians(y2)) 
                    * math.cos(math.radians(x1) - math.radians(x2)))
    return distance


df_tmp = pd.merge(df_store[['store_cd', 'address', 'longitude', 'latitude']],
                  df_customer_1[['store_cd', 'address_m', 'customer_id', 'longitude_m', 'latitude_m']], how='inner', on='store_cd')
df_tmp['dist'] = df_tmp[['longitude', 'latitude', 'longitude_m', 'latitude_m']].apply(lambda x:calc_distance(x[0], x[1], x[2], x[3]), axis=1)
df_tmp.head()

,store_cd,address,longitude,latitude,address_m,customer_id,longitude_m,latitude_m,dist
0,S12014,千葉県千葉市稲毛区千草台一丁目,140.118,35.63559,千葉県千葉市稲毛区轟町**********,CS014313000066,140.11014,35.62543,1.334520
1,S12014,千葉県千葉市稲毛区千草台一丁目,140.118,35.63559,千葉県千葉市稲毛区穴川**********,CS014515000254,140.10535,35.63266,1.188750
2,S12014,千葉県千葉市稲毛区千草台一丁目,140.118,35.63559,千葉県千葉市稲毛区天台**********,CS014515000067,140.11458,35.63179,0.523519
3,S12014,千葉県千葉市稲毛区千草台一丁目,140.118,35.63559,千葉県千葉市稲毛区天台**********,CS014313000092,140.11458,35.63179,0.523519
4,S12014,千葉県千葉市稲毛区千草台一丁目,140.118,35.63559,千葉県千葉市稲毛区萩台町**********,CS014513000138,140.12277,35.64254,0.884894


In [100]:
df_tmp=df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_customer_u=pd.merge(df_tmp, df_customer, how='inner', on='customer_id').sort_values(['amount', 'customer_id'], ascending=[False, True])
df_customer_u.drop_duplicates(['amount', 'customer_id'], keep='first', inplace=True)
df_customer_u.head()

,customer_id,amount,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
3899,CS017415000097,23086,福士 千夏,1,女性,1973-04-03,45,166-0014,東京都杉並区松ノ木**********,S13017,20151209,F-20101006-F
3488,CS015415000185,20153,岩淵 はるみ,1,女性,1973-09-19,45,135-0043,東京都江東区塩浜**********,S13015,20150322,F-20101014-F
6502,CS031414000051,19202,長澤 沙知絵,1,女性,1973-04-25,45,151-0064,東京都渋谷区上原**********,S13031,20150823,F-20101009-F
5908,CS028415000007,19127,紺野 あい,1,女性,1969-07-28,49,246-0023,神奈川県横浜市瀬谷区阿久和東**********,S14028,20151212,F-20100922-F
332,CS001605000009,18925,安部 耕司,0,男性,1952-10-22,66,144-0035,東京都大田区南蒲田**********,S13001,20160203,F-20101019-E


In [101]:
df_customer_n = pd.merge(df_customer, 
                         df_customer_u[['customer_name', 
                                        'postal_cd', 'customer_id']],
                         how='inner', on =['customer_name', 'postal_cd'])
df_customer_n.rename(columns={'customer_id_x':'customer_id', 
                              'customer_id_y':'integration_id'}, inplace=True)


In [102]:
del df_customer_n

In [103]:
del df_customer_1


In [104]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_sales = pd.merge(df_customer, df_tmp['customer_id'], how='inner', on='customer_id')
df_train, df_test = train_test_split(df_sales, random_state=71, test_size=0.2)
print(len(df_train), len(df_test))

6644 1662


In [111]:
df_tmp = df_receipt[['sales_ymd', 'amount']].copy()
df_tmp['sales_ym'] = df_tmp['sales_ymd'].astype(str).str[0:6]
df_tmp.groupby('sales_ym').agg({'amount':'sum'}).reset_index()
df_tmp.head()

def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = slide_window * start_point
    test_start = train_start + train_size
    return df[train_start: test_start], df[test_start: test_start + test_size]

df_train_1, df_test_1 = split_data(df_tmp, 12, 6, 6, 0)
df_train_2, df_test_2 = split_data(df_tmp, 12, 6, 6, 1)
df_train_3, df_test_3 = split_data(df_tmp, 12, 6, 6, 2)

In [112]:
df_train_1

,sales_ymd,amount,sales_ym
0,20181103,158,201811
1,20181118,81,201811
2,20170712,170,201707
3,20190205,25,201902
4,20180821,90,201808
5,20190605,138,201906
6,20181205,30,201812
7,20190922,128,201909
8,20170504,770,201705
9,20191010,680,201910


In [122]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')
df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

print('0の件数', len(df_tmp.query('buy_flg == 0')))
print('1の件数', len(df_tmp.query('buy_flg == 1')))

positive_count = len(df_tmp.query('buy_flg == 1'))

rs = RandomUnderSampler(random_state=71)

df_sample, _ = rs.fit_resample(df_tmp, df_tmp.buy_flg)

print('0の件数', len(df_sample.query('buy_flg == 0')))
print('1の件数', len(df_sample.query('buy_flg == 1')))


0の件数 13665
1の件数 8306
0の件数 8306
1の件数 8306


In [125]:
df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_s = df_customer.drop(columns='gender')
df_customer_s

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
...,...,...,...,...,...,...,...,...,...,...
21966,CS002512000474,市村 夏希,1,1959-10-12,59,185-0034,東京都国分寺市光町**********,S13002,20171110,0-00000000-0
21967,CS029414000065,上村 怜奈,1,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F
21968,CS012403000043,堀越 明,0,1972-12-16,46,231-0825,神奈川県横浜市中区本牧間門**********,S14012,20150406,0-00000000-0
21969,CS033512000184,池谷 華子,1,1964-06-05,54,245-0016,神奈川県横浜市泉区和泉町**********,S14033,20160206,0-00000000-0


In [126]:
df_product_full = pd.merge(df_product, df_category[['category_small_cd', 
                                                    'category_major_name',
                                                    'category_medium_name',
                                                    'category_small_name']], 
                           how = 'inner', on = 'category_small_cd')
df_product_full.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,04,0401,040101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,04,0401,040101,268.0,201.0,惣菜,御飯類,弁当類


In [136]:
df_product_full.to_csv('../data/P_df_product_full_UTF-8_header.csv', 
                       encoding='UTF-8', index=False)

In [129]:
# コード例2（BOM付きでExcelの文字化けを防ぐ）
df_product_full.to_csv('../data/P_df_product_full_UTF-8_header.csv', 
                       encoding='utf_8_sig', 
                       index=False)

In [131]:
df_product_full.to_csv('../data/P_df_product_full_CP932_header.csv', 
                       encoding='CP932', index=False)

In [132]:
df_product_full.to_csv('../data/P_df_product_full_UTF-8_noh.csv', 
                       header=False ,encoding='UTF-8', index=False)

In [137]:
df_tmp = pd.read_csv('../data/P_df_product_full_UTF-8_header.csv')
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


In [138]:
df_tmp = pd.read_csv('../data/P_df_product_full_UTF-8_header.csv', header=None)
df_tmp.head(10)

,0,1,2,3,4,5,6,7,8
0,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
1,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
2,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
3,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類
4,P040101004,04,0401,040101,248.0,186.0,惣菜,御飯類,弁当類
5,P040101005,04,0401,040101,268.0,201.0,惣菜,御飯類,弁当類
6,P040101006,04,0401,040101,298.0,224.0,惣菜,御飯類,弁当類
7,P040101007,04,0401,040101,338.0,254.0,惣菜,御飯類,弁当類
8,P040101008,04,0401,040101,420.0,315.0,惣菜,御飯類,弁当類
9,P040101009,04,0401,040101,498.0,374.0,惣菜,御飯類,弁当類


In [139]:
df_product_full.to_csv('../data/P_df_product_full_UTF-8_header.tsv', 
                       sep='\t', encoding='UTF-8', index=False)

In [141]:
df_tmp = pd.read_table('../data/P_df_product_full_UTF-8_header.tsv')
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


In [143]:
df_tmp = pd.read_csv('../data/P_df_product_full_UTF-8_header.tsv')
df_tmp.head(10)

,product_cd\tcategory_major_cd\tcategory_medium_cd\tcategory_small_cd\tunit_price\tunit_cost\tcategory_major_name\tcategory_medium_name\tcategory_small_name
0,P040101001\t04\t0401\t040101\t198.0\t149.0\t惣菜...
1,P040101002\t04\t0401\t040101\t218.0\t164.0\t惣菜...
2,P040101003\t04\t0401\t040101\t230.0\t173.0\t惣菜...
3,P040101004\t04\t0401\t040101\t248.0\t186.0\t惣菜...
4,P040101005\t04\t0401\t040101\t268.0\t201.0\t惣菜...
5,P040101006\t04\t0401\t040101\t298.0\t224.0\t惣菜...
6,P040101007\t04\t0401\t040101\t338.0\t254.0\t惣菜...
7,P040101008\t04\t0401\t040101\t420.0\t315.0\t惣菜...
8,P040101009\t04\t0401\t040101\t498.0\t374.0\t惣菜...
9,P040101010\t04\t0401\t040101\t580.0\t435.0\t惣菜...
